This notebook is structured to read levels from an xml and perform sample calculations. We start off by installing any missing packages this notebook will need to fully run.

In [ ]:
import sys, subprocess, pkg_resources
required = {'numpy','lvlspy','matplotlib'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed
if missing:
    subprocess.check_call([sys.executable, '-m','pip','install','--quiet',*missing])


import numpy as np
import lvlspy.level as lv
import lvlspy.spcoll as lc
import lvlspy.species as ls
import lvlspy.transition as lt
import matplotlib.pyplot as plt

You can also create new empty species collection, validate any existing xml, update the created empty collection and output to a specific unit

In [ ]:
#Empty collection
new_coll = lc.SpColl()

#Validate existing xml
new_coll.validate('example.xml')

#Update collection from xml
new_coll.update_from_xml('example.xml')

#Writing to new xml
new_coll.write_to_xml("new_example.xml")

#Write with specific units
new_coll.write_to_xml("new_example_ev.xml", units = 'ev')

!cat new_example_ev.xml

Let's extract the species from the collection and set a temperature (in Kelvin)

In [ ]:
sp = new_coll.get()['my_species'] #my_species should be replaced with whatever element you are testing, such as kr85 or ta180
T = 1.e+9 #K

From the stored properties in the xml, energies, Einstein A coefficients, and multiplicities, with the supplied temperature, we can calculate the rate matrix

In [ ]:
rate_matrix = sp.compute_rate_matrix(T)

print('\nRate Matrix:\n')

for i in range(rate_matrix.shape[0]):
    for j in range(rate_matrix.shape[1]):
        print(i, j, rate_matrix[i, j])

The following exercises can be done by going through the steps of the other notebook without having to import an xml, but for the sake of flexing the power of the API, this is the route we chose. Since we have a rate matrix, we can now evolve our species with time at a fixed temperature. The governing differential equation is $\frac{dY}{dt} = \Lambda Y$. The integrator for this 5 coupled equations is the Newton-Raphson method, a full description can be found at [webnucleo.org](https://webnucleo.readthedocs.io/en/latest/jupyter_notebooks.html).

The following definition calculates the vector f in the equation $A\delta = -f$. The following contains the setup and loop for the Newton-Raphson solver. If you are unfamiliar with the method, it is fully described 

In [ ]:
def f_vector(y_dt,y_i,A):
    return np.matmul(A,y_dt) - y_i

The following sets up and implements the Newton-Raphson method.

In [ ]:
tol = 1.e-6 #convergence tolerance on newton raphson solver 
t = np.linspace(0,1,1000) #Time array
dt = t[1] - t[0] #Time step
y = np.array([1.,0,0,0,0]) #initial condition where only the ground state is populated

A = np.identity(5) - dt*rate_matrix #since the matrix is constant in time, we calculate it once for time/efficiency sake

#A good guess of a NR solver is the value of the system at current time  
y_dt = y
###########################
#for memory and processing consideration, it is best to setup the 2D array that will contain the solution of the system 
#throughout time
Y = np.empty((5,len(t)))
#setting the 1st column to the initial condition
Y[:,0] = y

for i in range(1,len(t)): 
    delta = np.ones(5)
    while max(delta) > tol:
        delta = np.linalg.solve(A,-f_vector(y_dt,y,A))
        y_dt = y_dt + delta
    #updating the 2D array after convergence
    Y[:,i] = y_dt
    #setting the new initial condition to the converged solution from previous step
    y = y_dt

Before plotting the solution, we can also compute the equilibrium solution beforehand and see if our evolved solutions will tend to them.

In [ ]:
eq_probs = sp.compute_equilibrium_probabilities(T)

plt.figure(figsize=(12,12))
for i in range(len(sp.get_levels())):
    plt.plot(t,Y[i,:],label = str(i) + ' level')
        
for i in range(len(eq_probs)):
    plt.plot(1, eq_probs[i], 'x')

plt.legend()
plt.show()

The fugacity evolution of each level can be calculated based on the attained solution

In [ ]:
fugacities = np.empty((Y.shape[0], Y.shape[1]))

for i in range(len(sp.get_levels())):
    fugacities[i, :] = Y[i, :] / eq_probs[i]
    
plt.figure(figsize=(12,12))
for i in range(len(sp.get_levels())):
    plt.plot(t,fugacities[i,:],label = str(i) + ' level')
    
plt.xlim([0,0.1])